In [38]:
!pip install nbimporter

In [39]:
!pip install ipynb

In [40]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from pathlib import Path
import pickle
import nbimporter
import torch.nn.functional as F

In [41]:
# Import architectures
from recommenders_architecture import *

### Load data

In [42]:
# ======= Load Pairwise Training Data =======
current_dir = Path.cwd()

pairwise_data_train_path= current_dir.parent / "data" / "pairwise"/"pairwise_train.csv"
train = pd.read_csv(pairwise_data_train_path)
pairwise_data_val_path= current_dir.parent / "data" / "pairwise"/"pairwise_val.csv"
val = pd.read_csv(pairwise_data_val_path)


# ======= Load Item Metadata (1027-dim vectors) =======

encoded_dir = current_dir.parent / "data" / "encoded" / "per_feature"


In [43]:

# ======= Configurations =======
EMBEDDING_DIM = 24  # User embedding size
ITEM_FEATURE_DIM = 768
BATCH_SIZE = 512
EPOCHS = 10
LR = 0.00001  # Learning rate
VAL_SPLIT = 0.1

In [44]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
# with open(auto_encoder_metadata_file, 'rb') as f:
#     item_metadata = pickle.load(f)

In [92]:

# item_embeddings_tensor = torch.zeros(len(item_metadata), item_metadata[0].size(0))
# for idx, (item_id, embed) in enumerate(item_metadata.items()):
#     item_embeddings_tensor[idx] = embed


In [93]:
# auto_encoder_metadata_file_pt_out = encoded_dir / "compressed_all_data_encodings_256.pt"

In [94]:
# torch.save(item_embeddings_tensor,auto_encoder_metadata_file_pt_out )

##### Create concat of image and text

In [47]:
# import torch

# # ======= Create Combined Item Metadata =======
# item_metadata = {}

# for item_id in text_embeddings.keys():
#     text_embed = text_embeddings[item_id]  # (1027,)
#     image_embed = images_embeddings.get(item_id, torch.zeros(2048))  # (2048,) default to zeros if missing

#     # Concatenate along the feature dimension
#     combined_embed = torch.cat([text_embed, image_embed], dim=0)  # (3075,)
#     item_metadata[item_id] = combined_embed

# # Save the combined metadata dictionary
# metadata_save_path = encoded_dir / "item_metadata_text_image.pt"
# torch.save(item_metadata, metadata_save_path)

# print(f"✅ Item metadata saved to {metadata_save_path} with {len(item_metadata)} items.")


✅ Item metadata saved to /storage/yahlly/RecSys/data/encoded/item_metadata_text_image.pt with 198771 items.


In [82]:
a = np.load(os.path.join(encoded_dir, "normalized_floats.npy"))

In [83]:
print(a[:30])

[[0.8        0.00949246]
 [0.925      0.05608209]
 [0.7        0.00385279]
 [0.85       0.00590978]
 [0.875      0.00474326]
 [0.875      0.02966468]
 [0.9        0.        ]
 [0.85       0.0058712 ]
 [0.775      0.00856933]
 [0.925      0.        ]
 [0.875      0.00771151]
 [0.8        0.00800833]
 [0.925      0.00533691]
 [0.85       0.00593056]
 [0.925      0.00474326]
 [0.9        0.00889881]
 [0.825      0.00741469]
 [0.75       0.00800833]
 [0.925      0.        ]
 [0.925      0.00296231]
 [0.925      0.00533691]
 [0.8        0.01067976]
 [0.8        0.01184034]
 [0.925      0.        ]
 [0.85       0.00533691]
 [0.875      0.00593056]
 [0.775      0.01216388]
 [0.7        0.00504009]
 [0.9        0.01431586]
 [0.85       0.        ]]


In [104]:
import numpy as np
import os

# List of feature files
feature_list = ["normalized_floats.npy", "features_embeddings.npy", "title_embeddings.npy","main_category_embeddings.npy","details_embeddings.npy"]

# Load and store each feature embedding array
embeddings_list = []
for feature in feature_list:
    file_path = os.path.join(encoded_dir, feature)
    feature_item_metadata = np.load(file_path)
    embeddings_list.append(feature_item_metadata)

# Concatenate feature arrays along the feature dimension (axis=1)
# Assuming that each embedding array has shape (num_items, feature_dim)
item_metadata = np.concatenate(embeddings_list, axis=1)

# Optionally, inspect the shape or content
print("Embeddings shape:", item_metadata.shape)


Embeddings shape: (220890, 3074)


#### make the item metadata a tensor

In [105]:
import torch

# Convert the numpy array to a PyTorch tensor
item_embeddings_tensor = torch.from_numpy(item_metadata)

# Optionally, verify the shape of the resulting tensor
print("Item embeddings tensor shape:", item_embeddings_tensor.shape)

Item embeddings tensor shape: torch.Size([220890, 3074])


In [106]:
item_embeddings_tensor = torch.from_numpy(item_metadata).float().to(device)


In [ ]:
train

In [95]:
import pandas as pd
from pathlib import Path

# Define the path to the CSV file
user_item_file_path = current_dir.parent / "data" / "data_and_test_files" / "user_item_rating_table_train_with_idx.csv"

# Load the CSV file into a DataFrame
df2 = pd.read_csv(user_item_file_path)

# Create a dictionary mapping each user to a list of item IDs they interacted with.
# Adjust the column names ('user_id' and 'item_id') if your CSV uses different names.
#user_interacted_items = df2.groupby('user_idx')['item_idx'].apply(list).to_dict()
train_interacted_items = train.groupby('user_id')['label'].apply(list).to_dict()
users_list = sorted(df2['user_idx'].unique())
# Ensure every user in your overall users_list appears in the dictionary.
# If a user doesn't appear in the train interactions, assign an empty list.
user_interacted_items = {user: train_interacted_items.get(user, []) for user in users_list}
print("Number of users in user_interacted_items:", len(user_interacted_items))

Number of users in user_interacted_items: 1096901


### Data loader class

In [96]:
# ======= Custom Dataset Class =======
class PairwiseDataset(Dataset):
    def __init__(self, dataframe):
        self.users = dataframe["user_id"].values
        self.item1 = dataframe["item1_id"].values
        self.item2 = dataframe["item2_id"].values
        self.labels = dataframe["label"].values

    def __len__(self):
        return len(self.users)

    def __getitem__(self, idx):
        return (
            self.users[idx],
            self.item1[idx],
            self.item2[idx],
            self.labels[idx],
        )


In [97]:
train

,user_id,item1_id,item2_id,label,timestamp,rating
0,0,0,13349,0,1349041740000,5.0
1,0,22959,1,1,1370958618000,1.0
2,0,97562,2,2,1440038761000,5.0
3,0,23003,3,3,1483320893000,3.0
4,1,179127,5,5,1600753653091,5.0
...,...,...,...,...,...,...
8030471,1096899,26803,32852,32852,1692552324736,5.0
8030472,1096899,177842,10643,10643,1692552357767,5.0
8030473,1096900,183765,86867,183765,1600792118191,1.0
8030474,1096900,155119,99585,155119,1615811081145,1.0


In [98]:
max_interactions

1438

In [107]:
import torch

max_allowed_interactions = 32

# Build padded_user_interactions and user_interaction_mask with a maximum of 256 interactions per user.
num_users = max(user_interacted_items.keys()) + 1  # or len(user_interacted_items) if keys are consecutive

# Create tensors of fixed size (256) for all users.
padded_user_interactions = torch.full((num_users, max_allowed_interactions), fill_value=-1, dtype=torch.long)
user_interaction_mask = torch.zeros((num_users, max_allowed_interactions), dtype=torch.float)

for uid, items in user_interacted_items.items():
    # Limit interactions to at most 256 items (you can use the first 256 or sample randomly)
    trimmed_items = items[:max_allowed_interactions]
    num_items = len(trimmed_items)
    padded_user_interactions[uid, :num_items] = torch.tensor(trimmed_items, dtype=torch.long)
    user_interaction_mask[uid, :num_items] = 1.0

# Optionally, move them to your device:
padded_user_interactions = padded_user_interactions.to(device)
user_interaction_mask = user_interaction_mask.to(device)


In [108]:
class TwoTowerModelBertFeatures(nn.Module):
    def __init__(self, num_users, num_items, embedding_dim, item_metadata_dim):
        super(TwoTowerModel, self).__init__()
        
        # User Tower (Processes item metadata of past interactions)
        self.user_fc = nn.Sequential(
            nn.Linear(item_metadata_dim, 512),
            nn.ReLU(),
            nn.Linear(512, embedding_dim),
        )
        # Item Tower (Processes item metadata)
        self.item_fc = nn.Sequential(
            nn.Linear(item_metadata_dim, 512),
            nn.ReLU(),
            nn.Linear(512, embedding_dim),
        )

    def forward(self, user_ids, item1_ids, item2_ids):
        user_ids = user_ids.long().to(device)
        item1_ids = item1_ids.to(device)
        item2_ids = item2_ids.to(device)
        
        # Retrieve the precomputed interactions and mask for each user in the batch
        batch_interactions = padded_user_interactions[user_ids]  # (batch_size, 256)
        batch_mask = user_interaction_mask[user_ids]            # (batch_size, 256)
        
        # Replace padded indices (-1) with 0 (or any valid index) before lookup.
        valid_batch_interactions = batch_interactions.clone()
        valid_batch_interactions[valid_batch_interactions < 0] = 0
        
        # Gather item metadata for all interactions in the batch.
        # item_embeddings_tensor has shape (num_items, item_metadata_dim)
        batch_item_metadata = item_embeddings_tensor[valid_batch_interactions]  # (batch_size, 256, item_metadata_dim)
        
        # Process the metadata through the user tower.
        processed = self.user_fc(batch_item_metadata)  # (batch_size, 256, embedding_dim)
        
        # Zero out padded positions using the mask.
        batch_mask_expanded = batch_mask.unsqueeze(-1)  # (batch_size, 256, 1)
        processed = processed * batch_mask_expanded
        
        # Compute the mean for valid interactions.
        user_embed = processed.sum(dim=1) / (batch_mask.sum(dim=1, keepdim=True) + 1e-8)
        
        # Process item towers.
        item1_embed = self.item_fc(item_embeddings_tensor[item1_ids])
        item2_embed = self.item_fc(item_embeddings_tensor[item2_ids])
        
        return user_embed, item1_embed, item2_embed
# ======= Pairwise BPR Loss =======


class BPRLoss(nn.Module):
    def __init__(self):
        super(BPRLoss, self).__init__()

    def forward(self, user_embed, item1_ids, item1_embed, item2_ids, item2_embed, labels):
        """
        Compute Bayesian Personalized Ranking (BPR) loss.

        Args:
        - user_embed: Tensor of shape (batch_size, embed_dim), user embeddings.
        - item1_ids: Tensor of shape (batch_size,), IDs of item1.
        - item1_embed: Tensor of shape (batch_size, embed_dim), embeddings for item1.
        - item2_ids: Tensor of shape (batch_size,), IDs of item2.
        - item2_embed: Tensor of shape (batch_size, embed_dim), embeddings for item2.
        - labels: Tensor of shape (batch_size,), IDs of the correct (positive) item.

        Returns:
        - loss: Computed BPR loss.
        """
        # Convert labels to binary: 1 if item1 is the positive item, else 0
        labels_binary = (labels == item1_ids).float()

        # Compute scores
        score1 = (user_embed * item1_embed).sum(dim=1)  # Affinity score for item1
        score2 = (user_embed * item2_embed).sum(dim=1)  # Affinity score for item2

        # Assign correct positive and negative scores based on labels_binary
        pos_score = torch.where(labels_binary == 1, score1, score2)
        neg_score = torch.where(labels_binary == 1, score2, score1)

        # Compute BPR loss
        loss = -F.logsigmoid(pos_score - neg_score).mean()
        return loss


In [20]:
df.head()

,user_id,item1_id,item2_id,label
0,0,50787,0,0
1,0,1,33482,1
2,0,35983,2,2
3,0,3,125581,3
4,0,4,60296,4


In [61]:
train_loader = DataLoader(PairwiseDataset(train), batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(PairwiseDataset(val), batch_size=BATCH_SIZE, shuffle=False)

In [62]:
num_users = 1096901
num_items = 198771


In [109]:
ITEM_FEATURE_DIM = 3074

In [103]:

# ======= Initialize Model, Loss, Optimizer =======
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TwoTowerModel(num_users, num_items, EMBEDDING_DIM, ITEM_FEATURE_DIM).to(device)
criterion = BPRLoss()
optimizer = optim.Adam(model.parameters(), lr=LR*10)

# ======= Training Loop =======
# ======= Training & Validation =======
log_file = "cold_training_log.txt"

print("🚀 Training Model...")
with open(log_file, "w") as log:
    log.write("🚀 Training Model...\n")
    for epoch in range(EPOCHS):
        model.train()
        train_loss = 0
    
        for user_ids, item1_ids, item2_ids,labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
            user_ids, item1_ids, item2_ids,labels = user_ids.to(device), item1_ids.to(device), item2_ids.to(device), labels.to(device)
    
            # Forward Pass
            user_embed, item1_embed, item2_embed = model(user_ids, item1_ids, item2_ids)
            #print(item1_embed==item2_embed)
            # Compute Loss
            # print(item1_embed==item2_embed)
            loss = criterion(user_embed,item1_ids, item1_embed,item2_ids, item2_embed, labels)
            train_loss += loss.item()
            
            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        train_loss = train_loss /len(train_loader)
        # ======= Validation =======
        model.eval()
        correct = 0
        total = 0
        val_loss = 0
        for user_ids, item1_ids, item2_ids, labels in val_loader:
            user_ids, item1_ids, item2_ids, labels = (
                user_ids.to(device),
                item1_ids.to(device),
                item2_ids.to(device),
                labels.to(device),
            )
            user_embed, item1_embed, item2_embed = model(user_ids, item1_ids, item2_ids)
            #print((item1_embed==item2_embed).all())
            score1 = (user_embed * item1_embed).sum(dim=1)  # Score for item1
            score2 = (user_embed * item2_embed).sum(dim=1)  # Score for item2
    
            # Determine the correct positive and negative scores based on labels
            labels_binary = (labels == item1_ids).float()
            #print(labels_binary)
            pos_scores = torch.where(labels_binary == 1, score1, score2)
            neg_scores = torch.where(labels_binary == 1, score2, score1)
            #print(pos_scores)
            # Check if the model correctly ranked the positive item higher
            loss = criterion(user_embed,item1_ids, item1_embed,item2_ids, item2_embed, labels)
            val_loss += loss.item()
            predictions = pos_scores > neg_scores
    
            correct += predictions.sum().item()
            total += predictions.shape[0]
    
        val_accuracy = correct / total
        val_loss=val_loss/len(val_loader)
        print(f"Epoch {epoch+1}: Train Loss = {train_loss:.4f},Val Loss = {val_loss:.4f}, Val Accuracy = {val_accuracy:.4f}")
        log.write(f"Epoch {epoch+1}: Train Loss = {train_loss:.4f}, Val Loss = {val_loss:.4f}, Val Accuracy = {val_accuracy:.4f}\n")



# ======= Save Model =======
#torch.save(model.state_dict(), "trained_model.pth")
print("✅ Model Training Complete!")
with open(log_file, "a") as log:
    log.write("✅ Model Training Complete!\n")

🚀 Training Model...


Epoch 1/10: 100%|██████████| 15685/15685 [01:20<00:00, 195.64it/s]


Epoch 1: Train Loss = 0.6549,Val Loss = 0.7651, Val Accuracy = 0.4979


Epoch 2/10:   3%|▎         | 548/15685 [00:03<01:31, 165.75it/s]


KeyboardInterrupt: 

### Optuna hyperparameter Tuning

In [112]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from tqdm import tqdm
import optuna
import wandb

# Assume these are defined elsewhere: TwoTowerModel, BPRLoss, train_loader, val_loader,
# padded_user_interactions, user_interaction_mask, item_embeddings_tensor,
# num_users, num_items, ITEM_FEATURE_DIM, and device

# Define the objective function for Optuna
def objective(trial):
    # Sample hyperparameters
    lr = trial.suggest_float("lr", 1e-5, 1e-2, log=True)
    embedding_dim = trial.suggest_int("embedding_dim", 16, 128)
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-2, log=True)
    epochs = 20  # fixed for now
    
    # Initialize Weights & Biases for monitoring
    wandb.init(project="TwoTowerModel_tuning", config={
        "lr": lr,
        "embedding_dim": embedding_dim,
        "weight_decay": weight_decay,
        "epochs": epochs,
    }, reinit=True)
    
    # Initialize model, loss, optimizer with sampled hyperparameters
    model = TwoTowerModel(num_users, num_items, embedding_dim, ITEM_FEATURE_DIM).to(device)
    criterion = BPRLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    
    best_val_loss = float("inf")
    
    # Training & validation loop
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        
        for user_ids, item1_ids, item2_ids, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
            user_ids = user_ids.to(device)
            item1_ids = item1_ids.to(device)
            item2_ids = item2_ids.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            user_embed, item1_embed, item2_embed = model(user_ids, item1_ids, item2_ids)
            loss = criterion(user_embed, item1_ids, item1_embed, item2_ids, item2_embed, labels)
            train_loss += loss.item()
            loss.backward()
            optimizer.step()
        train_loss /= len(train_loader)
        
        # Validation
        model.eval()
        val_loss = 0
        correct = 0
        total = 0
        with torch.no_grad():
            for user_ids, item1_ids, item2_ids, labels in val_loader:
                user_ids = user_ids.to(device)
                item1_ids = item1_ids.to(device)
                item2_ids = item2_ids.to(device)
                labels = labels.to(device)
                
                user_embed, item1_embed, item2_embed = model(user_ids, item1_ids, item2_ids)
                loss = criterion(user_embed, item1_ids, item1_embed, item2_ids, item2_embed, labels)
                val_loss += loss.item()
                
                # Calculate scores and accuracy
                score1 = (user_embed * item1_embed).sum(dim=1)
                score2 = (user_embed * item2_embed).sum(dim=1)
                labels_binary = (labels == item1_ids).float()
                pos_scores = torch.where(labels_binary == 1, score1, score2)
                neg_scores = torch.where(labels_binary == 1, score2, score1)
                predictions = pos_scores > neg_scores
                correct += predictions.sum().item()
                total += predictions.shape[0]
        val_loss /= len(val_loader)
        val_accuracy = correct / total
        
        # Log metrics to wandb
        wandb.log({
            "epoch": epoch+1,
            "train_loss": train_loss,
            "val_loss": val_loss,
            "val_accuracy": val_accuracy,
        })
        
        # Report intermediate objective value to Optuna
        trial.report(val_loss, epoch)
        if trial.should_prune():
            wandb.finish()
            raise optuna.exceptions.TrialPruned()
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
    
    wandb.finish()
    return best_val_loss

# Create and run the Optuna study
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20)

print("Best trial:")
trial = study.best_trial
print(trial.params)


[I 2025-03-08 11:07:17,578] A new study created in memory with name: no-name-e640cf32-022f-4514-b1b1-1b822d320c46


Epoch 20/20: 100%|██████████| 15685/15685 [02:41<00:00, 96.83it/s]


epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
train_loss,▁▁▂▂▃▄▆▅▅▅▄▅▆▆▇▇▇▇██
val_accuracy,▅▃▃▁▃▂█▂█▂▇▅▅▅▂██▇▆▆
val_loss,▁▁▁▁▁▁▁▁▁▁▁▃▁▁▁▁▁█▁▁
epoch,20
train_loss,34.20171
val_accuracy,0.50002
val_loss,0.72475


[I 2025-03-08 12:06:08,355] Trial 0 finished with value: 0.6941605195563028 and parameters: {'lr': 0.0033745896111542595, 'embedding_dim': 86, 'weight_decay': 8.956962633318771e-06}. Best is trial 0 with value: 0.6941605195563028.
wandb: Currently logged in as: ronelias (ronelias-tel-aviv-university). Use `wandb login --relogin` to force relogin


Epoch 20/20: 100%|██████████| 15685/15685 [02:43<00:00, 95.93it/s]


...

[I 2025-03-08 21:22:08,050] Trial 18 finished with value: 0.6931470849777681 and parameters: {'lr': 0.003997713255968757, 'embedding_dim': 115, 'weight_decay': 0.0019720856481734218}. Best is trial 12 with value: 0.6931438477474342.


Epoch 1/20: 100%|██████████| 15685/15685 [02:42<00:00, 96.79it/s]


epoch,▁
train_loss,▁
val_accuracy,▁
val_loss,▁
epoch,1
train_loss,0.63389
val_accuracy,0.5012
val_loss,0.76087


[I 2025-03-08 21:25:14,212] Trial 19 pruned. 


Best trial:
{'lr': 0.009254651603008139, 'embedding_dim': 103, 'weight_decay': 0.009452317834358981}


##### PREDICT ON COLD ITEMS